In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    
}


In [3]:
df = pd.DataFrame()
a = 0

#sayfa sayısı kadar döndürür (4.sayfaya kadar döndürdü)
for pageNumber in range(1, 31):
    r = requests.get(f"https://www.kariyer.net/is-ilanlari/bilisim-{pageNumber}", headers=header)
    soup = BeautifulSoup(r.content, "lxml")
    jobNotices  = soup.find_all("div", attrs={"class":"list-items"})

    time.sleep(5)

    for notice in jobNotices:

        # Sayfadan ilan linkleri oluşturma
        url = notice.a.get("jobid")

        link = f"https://api.kariyer.net/job?jobId={url}&memberId=30529505"

        response = requests.get(link, headers=header)

        data = response.json()
        try:
            # Json dosyasından sayfa kriterlerini çekiyoruz
            companyName= data["data"]["jobCompanyInformation"]["companyName"]
            companyInfo = data["data"]["jobCompanyInformation"]["companyInfo"].strip()
            location = data["data"]["jobGeneralInformation"]["locationText"]
            title = data["data"]["jobGeneralInformation"]["title"]
            qualifications = data["data"]["jobGeneralInformation"]["qualifications"]
            totalApplication = data["data"]["jobIstatistics"]["totalApplication"]
            workType = data["data"]["jobPositionInformation"]["workTypeText"]
            positionName = data["data"]["jobPositionInformation"]["positionName"]
            positionLevel = data["data"]["jobPositionInformation"]["positionLevelText"]
            positionDepartment = data["data"]["jobPositionInformation"]["workAreas"][0]["name"]
            jobCandidateCriteria =  data["data"]["jobCandidateCriteria"]["educationLevelText"] #liste
            experience =  data["data"]["jobCandidateCriteria"]["experienceText"]
            

            #Her bir ilan için sözlük oluşturulur.
            job_dict = {"Şirket": companyName,
            "Şirket Tanımı" : companyInfo,
            "Şehir": location, 
            "Başlık": title, 
            "İş Tanımı": qualifications, 
            "Başvuru Sayısı": totalApplication, 
            "Çalışma Şekli" :  workType, 
            "Pozisyon İsmi": positionName, 
            "Pozisyon Seviyesi": positionLevel,
            "Departman": positionDepartment,
            "Kriterler" : jobCandidateCriteria,
            "Tecrube" : experience
            }

            #Oluşturulan sözlükler df ile birleştirilir.
            df_dictionary = pd.DataFrame([job_dict])
            df = pd.concat([df, df_dictionary], ignore_index=True)


        except:
            pass
    a+=1
    print(f'{a}.Sayfa Alındı')


1.Sayfa Alındı
2.Sayfa Alındı
3.Sayfa Alındı
4.Sayfa Alındı
5.Sayfa Alındı
6.Sayfa Alındı
7.Sayfa Alındı
8.Sayfa Alındı
9.Sayfa Alındı
10.Sayfa Alındı
11.Sayfa Alındı
12.Sayfa Alındı
13.Sayfa Alındı
14.Sayfa Alındı
15.Sayfa Alındı
16.Sayfa Alındı
17.Sayfa Alındı
18.Sayfa Alındı
19.Sayfa Alındı
20.Sayfa Alındı
21.Sayfa Alındı
22.Sayfa Alındı
23.Sayfa Alındı
24.Sayfa Alındı
25.Sayfa Alındı
26.Sayfa Alındı


In [ ]:
df.head(5)

,Başlık,İş Tanımı,Şehir,Başvuru Sayısı,Çalışma Şekli,Pozisyon İsmi,Pozisyon Seviyesi,Departman,Kriterler,Tecrube,Şirket Tanımı
0,Kundendienst Mitarbeiter(İn),Wir sind ein wachsendes Start-Up Unternehmen i...,İzmir,24,Serbest,Teknik Destek Uzmanı,Uzman,Müşteri Hizmetleri / Çağrı Merkezi,[Üniversite(Mezun)],Tecrübeli / Tecrübesiz,Kariyer.net olarak 21 yıldır dijital teknoloji...
1,Depo Elemanı,Türkiye’nin En Büyük Online Kozmetik Butiği Ko...,İstanbul(Asya),66,Tam Zamanlı,Depo Elemanı,Eleman,Depo / Antrepo,[İlk Öğretim(Mezun)],En az 1 yıl tecrübeli,Cevkoz Şirketlerinin temeli 1979 yılında atılm...
2,Java Yazılım Uzmanı,GENEL NİTELİKLERÜniversitelerin Bilgisayar/Yaz...,İstanbul(Asya),80,Tam Zamanlı,Java Yazılım Uzmanı,None,AR-GE,"[Üniversite(Mezun), Yüksek Lisans(Öğrenci), Do...",En az 3 yıl tecrübeli,Netaş: Türkiye’nin köklü ve en yetkin bilişim ...
3,Backend Yazılım Geliştirme Mühendisi,"GENEL NİTELİKLERÜniversitelerin Bilgisayar, El...",İstanbul(Asya),161,Tam Zamanlı,Yazılım Geliştirme Mühendisi,None,AR-GE,"[Üniversite(Mezun), Yüksek Lisans(Öğrenci), Yü...",Tecrübeli / Tecrübesiz,Netaş: Türkiye’nin köklü ve en yetkin bilişim ...
4,Müşteri Hizmetleri Temsilcisi,Bayraklı tower'da Profesyonel bir ofise sahip ...,İzmir,1010,Tam Zamanlı,Müşteri Hizmetleri Temsilcisi,Yeni Başlayan,E-Ticaret,"[Lise(Mezun), Üniversite(Mezun)]",Tecrübeli / Tecrübesiz,HAKKIMIZDA\r\n\r\nFirmamız 2007 yılında kurula...


In [5]:
df.shape

(1537, 11)

In [11]:
file_name = 'kariyer.xlsx'

df.to_excel(file_name)